# Creating a KNN Model

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn 
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import accuracy_score 
from matplotlib.pyplot import figure
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as skp

## Cleaning the dataset

In [2]:
df = pd.read_csv('features_3_sec.csv')

In [3]:
df.drop('filename', inplace=True, axis=1)
df.drop('length', inplace=True, axis=1)

In [4]:
df

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,8.479527e+05,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,1.170890e+06,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,2.602871e+06,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,4.968878e+05,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


## Allocating X and Y & Splitting the data

In [5]:
df_y = df["label"]
df_x = df.drop("label", axis = 1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42, stratify=df_y)


## Standardizing for more efficient classifier

In [7]:
standardizer = StandardScaler()

X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

## Training a random KNN without hyperparameterizing

In [8]:
knn = KNeighborsClassifier()

In [9]:
knn.fit(X_train, y_train)
print("Train Score:", knn.score(X_train, y_train))
print("Test Score:", knn.score(X_test, y_test))

Train Score: 0.9365615615615616
Test Score: 0.8773773773773774


## Grid Search and Hyperparameterizing with Cross Validation

In [10]:
grid_params = [{'n_neighbors': [3, 5, 10, 15, 20, 25],
               'weights': ['uniform', 'distance'],
               'metric':['euclidean', 'manhattan'], 
               'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}]

In [11]:
grid_search = GridSearchCV(KNeighborsClassifier(), grid_params, verbose=2, cv=10, n_jobs=-1)

In [12]:
grid_model = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


## Printing out the best parameters and Creating the best KNN

In [13]:
grid_model.best_params_

{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 3,
 'weights': 'distance'}

In [14]:
knn_cv = KNeighborsClassifier(n_neighbors = 3, weights = 'distance', metric = 'manhattan', algorithm = 'auto')
knn_cv.fit(X_train, y_train)


KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')

In [24]:
y_pred = knn_cv.predict(X_test)
print("Train Score:", knn_cv.score(X_train, y_train))
print("Test Score:", knn_cv.score(X_test, y_test))

Train Score: 0.998998998998999
Test Score: 0.9214214214214215
